In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sn
import plotly.express as px
import json
import datetime
#import geopandas as gpd
import fiona
import geoTim
import shapely
import pyproj

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

In [2]:
counts = pd.read_csv('C:\\Users\\comp\\Documents\\New York\\Traffic_Volume_Counts__2014-2018_.csv')

In [43]:
counts['total'] = counts.iloc[:,7:31].sum(axis = 1)

In [50]:
counts_date = pd.DataFrame(counts.groupby(['Segment ID', 'Date'])['total'].sum()).reset_index()
counts_date

,Segment ID,Date,total
0,202,10/11/2014,11459.0
1,1416,10/24/2015,9481.0
2,1416,10/25/2015,7851.0
3,1416,10/26/2015,9643.0
4,1416,10/27/2015,9718.0
...,...,...,...
14931,9017039,10/15/2014,20228.0
14932,9017039,10/16/2014,21281.0
14933,9017039,10/17/2014,19250.0
14934,9017039,10/18/2014,12740.0


In [54]:
counts_segid = pd.DataFrame(counts_date.groupby('Segment ID')['total'].mean()).reset_index()
counts_segid

,Segment ID,total
0,202,11459.000000
1,1416,9504.777778
2,1883,9513.000000
3,2147,10627.666667
4,2367,13352.111111
...,...,...
1485,9014615,34409.555556
1486,9014764,62537.555556
1487,9017020,19013.555556
1488,9017037,13595.333333


In [4]:
lion_path = 'C:\\Users\\comp\\Documents\\New York\\nyclion_19d\\lion\\lion.gdb'

layers = fiona.listlayers(lion_path)
print(layers)

lion = fiona.open(lion_path, 'r', layer = layers[3])

['node', 'node_stname', 'altnames', 'lion']


In [6]:
from shapely.geometry import asShape

features = []

for rec in lion:
    #print(rec)
    p = asShape(rec['geometry']).centroid
    features.append([rec['properties']['SegmentID'], p])
    
df_coords = pd.DataFrame(features, columns = ['SegmentID', 'geometry'])

In [25]:
#df_coords['SegmentID'] = df_coords['SegmentID'].astype(int) 
df_coords.to_csv('segment_coords.csv', index = False)

In [24]:
# df_coords.rename(columns = {'coords' : 'geometry'}, inplace = True)

In [22]:
np.shape(df_coords)

(228011, 2)

In [7]:
from functools import partial

transformer = pyproj.Transformer.from_crs("epsg:2263", "epsg:4326", always_xy = True)
project = partial(transformer.transform)

df_coords['geometry'] = df_coords['geometry'].apply(lambda x : geoTim.transform_multi(project, x))

In [55]:
df_merge = counts_segid.merge(df_coords, how = 'left', left_on = 'Segment ID', right_on = 'SegmentID')

In [56]:
df_merge['type'] = df_merge['geometry'].apply(lambda x : 'float' if type(x) == float else 'other')

In [57]:
df_merge.groupby('type').count()

,Segment ID,total,SegmentID,geometry
type,,,,
float,87,87,0,0
other,1634,1634,1634,1634


In [58]:
df_merge = df_merge[df_merge['type'] != 'float']

In [59]:
df_merge['x'] = df_merge['geometry'].apply(lambda p : p.x)
df_merge['y'] = df_merge['geometry'].apply(lambda p : p.y)

In [27]:
#gj = geoTim.df_to_geojson(df_merge)

In [60]:
df_merge.columns

Index(['Segment ID', 'total', 'SegmentID', 'geometry', 'type', 'x', 'y'], dtype='object')

In [61]:
token = open("C:\\Users\\comp\\Documents\\New York\\token").read()

fig = px.scatter_mapbox(df_merge, lon = df_merge['x'], lat = df_merge['y'], color = 'total', size = 'total')

fig.update_layout(mapbox_style="light", mapbox_accesstoken = token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()